In [1]:
import pandas as pd
import unicodedata
import os
import numpy as np
import math

In [2]:
curDir= os.path.abspath(os.getcwd())

if 'd:\\DATN\\model' not in curDir:
    curDir = 'd:\\DATN\\model'

# print(curDir)

Function loadData để tải các dữ liệu cần thiết từ file data.xls bao gồm các sheet: 
1. ruleGroups: tập hợp các nhóm luật, bao gồm điểm nhóm, mã nhóm, điều kiện
1. rules: các luật cụ thể của từng nhóm luật, bao gồm trọng số điểm, mã luật, điều kiện 
1. buyers: danh sách KH, và test case yêu cầu tương ứng
1. distance: khoảng cách giữa các quận 

In [3]:
### Load data
### Sử dụng để load dữ liệu bất động sản, dữ liệu các bộ rule, và dữ liệu KH
def loadData(isDebug=0):
    ruleFile= curDir + '\\data.xlsx'
    ### Load rules
    global rulesDf
    rulesDf= pd.read_excel(ruleFile, sheet_name = 'rules').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    if isDebug==1:
        print("Read rule list, total is: ",len(rulesDf))
    ### Load rule group
    global ruleGroupsDf
    ruleGroupsDf= pd.read_excel(ruleFile, sheet_name = 'ruleGroups').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    if isDebug==1:
        print("\nRead Rule groups, total is: ",len(ruleGroupsDf))
        print(ruleGroupsDf.head(1))
    ### Load buyer
    global buyersDf
    buyersDf= pd.read_excel(ruleFile, sheet_name = 'buyers').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    if isDebug==1:
        print("\nBuyer: ", len(buyersDf))
        print(buyersDf.head(1))
    ### load houses
    global housesDf
    housesDf= pd.read_excel(curDir +'\\houseAll.xls', sheet_name = 'house').apply(lambda x: x.str.strip() if x.dtype == "object" else x) # full data
    if isDebug==1:
        print("\nHouse: ",len(housesDf))
        print(housesDf.head(1))
    ### Load distance between district
    global distancesDf
    distancesDf= pd.read_excel(ruleFile, sheet_name = 'distance').apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    if isDebug==1:
        print('-----------------------\n')
    ### change DataFrame to Dictionary
    global rules
    rules= rulesDf.to_dict('records')
    if isDebug==1:
        print("Rule no 0: ", rules[0])
    global ruleGroups
    ruleGroups= ruleGroupsDf.to_dict('records')
    if isDebug==1:
        print("Rule group no 0: ", ruleGroups[0])
    global houses
    houses= housesDf.to_dict('records')
    if isDebug==1:
        print("House no 0: ", houses[0])
    global buyers
    buyers= buyersDf.to_dict('records')
    if isDebug==1:
        print("Buyer no 0: ", buyers[0])

loadData(isDebug=0)    

# 0. Common Functions

In [4]:
def isValidInput(x):
    if x is None or pd.isna(x) or str(x).strip()=='': #np.isnan(x) 
        return False
    else:
        return True

def isNotNoneInput(x):
    if x is None or np.isnan(x) or str(x).strip()=='':
        return False
    else:
        return True
    
def isGoodLegalStatus(x):
    if x is None or pd.isna(x) or str(x).strip()=='': #np.isnan(x) 
        return False
    if x in ['Sổ hồng', 'Sổ đỏ']:
        return True    
    return False

def getValue(x, d=0):
    if math.isnan(x):
        return d
    return x
# def isNumber(x):
#     if isValidInput(x) and

# 2. Hàm để tính toán điểm

In [5]:
# Hàm tính toán khoảng cách dựa trên 2 quận
def fn_calculateDistance(srcPlace, tgtPlace):
    res = distancesDf[ (distancesDf.district_name == srcPlace) & (distancesDf.district_name_2 == tgtPlace) ]
#     print(res)
    if len(res) == 0:
        return  10000000
    return round(res.iloc[0]["fn_calculate_distance"], 2)

# fn_calculateDistance('bình thạnh', 'quận 2')
# fn_calculateDistance('Bình Thạnh', 'Quận 2')

1. Hàm calculateSingleRuleScore: dựa trên thông tin KH, thông tin BĐS, và 1 luật dẫn cụ thể để xác định ngôi nhà có thoả điều kiện của luật dẫn cho khách hàng hay không 
1. Hàm calculateScoreOfHouse: đối với mỗi khách hàng và BĐS
- Duyệt từng nhóm luật và xác định nhóm luật có được thực thi không 
- Ứng với từng luật tính toán mức độ phù hợp của BĐS với KH
- Tổng hợp kết quả mức độ phù hợp của BĐS

In [6]:
### version 2
def calculateSingleRuleScore(buyerInfo, houseInfo, ruleInfo, **kwargs):
    ### get-set key-value from parent function
    for key, value in kwargs.items():
        locals()[key] = value
    ruleScore = ruleInfo.get("score")
    ### execute rule
    ruleExpression = unicodedata.normalize('NFKD', ruleInfo.get("rule_expression")).strip() 
    exprValue = eval(ruleExpression)
    # print(f'exprValue: {ruleScore}')
    if exprValue > 0:
        return ruleScore
    else:
        return 0
    
def calculateScoreOfHouse(buyerInfo, houseInfo, ruleDf, ruleGroupList, isDebugRule=0):
    pre_defined_kwargs = {}
    _calRuleGroup = []
    for ruleGroup in ruleGroupList:
        groupCondition = unicodedata.normalize('NFKD', ruleGroup.get("conditions")).strip()
        groupScore= float(ruleGroup.get("group_point"))
        groupCode= ruleGroup.get("rule_group")
        ruleGroupCode =  ruleGroup.get("rule_group")
        groupRes={'groupCode': groupCode, 'groupCondition': groupCondition, 'groupScore': groupScore}
        if isDebugRule==1:
            print("\tCode of group", ruleGroupCode, " | Group condition:", groupCondition)
        if eval(groupCondition) is False:
            if isDebugRule==1:
                print("Break rule code: ", ruleGroupCode)
        else:
            curRule= {}
            curScore= 0
            filteredRuleList = ruleDf[ruleDf['rule_group'] == ruleGroupCode].to_dict('records')
            ### iterate each rule in Group, if condition matches and higher score, set this rule as result for group
            for r in filteredRuleList:
                if isDebugRule==1:
                    print(r)
                _score = calculateSingleRuleScore(buyerInfo, houseInfo, r, **pre_defined_kwargs)
                if isDebugRule==1:
                    print(_score)
                if _score > 0 and _score > curScore:
                    curRule= r
                    curScore=_score
            groupRes.update(curRule)            
            groupRes.update({'calRuleScore': curScore*groupScore})
            _calRuleGroup.append(groupRes)
    # calculate total score
    _maxScore= 0
    _totalScore=0
    for c in _calRuleGroup:
        _maxScore+= c.get("groupScore", 0)
        _totalScore+= c.get("calRuleScore", 0)
    if isDebugRule==1:
        print('REPORT:')
        print('Total calculated Score: ',_totalScore)
        print('Max Score from Groups: ', _maxScore)
    _housePerf= 0.0
    if _maxScore != 0:
        _housePerf=round(_totalScore*100/_maxScore, 2)
    _res= {'executedRuleGroup': _calRuleGroup, 'houseInfo': houseInfo,
           'maxScore': _maxScore, 'houseScore': _totalScore, 'housePerformance':_housePerf,
           'buyerInfo': buyerInfo}
    _res.update(buyerInfo)
    _res.update(houseInfo)
    return _res

loadData()

c=11
print('Customer info: ', buyers[c],'\n-------------------')
for i in range(0,1):
    _houseEvaluation= calculateScoreOfHouse(buyers[c], houses[i], rulesDf, ruleGroups, isDebugRule= 1)
    # print(_houseEvaluation)
    print('-------------\n')

Customer info:  {'id': 'C1', 'group': 2.0, 'age': 40.0, 'is_married': 1.0, 'no_child': 2.0, 'month_income': 50000000.0, 'monthly_amt': 40000000.0, 'avai_amt': 1500000000.0, 'desired_location': 'Quận 7', 'desired_interiorStatus': nan} 
-------------------
	Code of group A  | Group condition: isValidInput( buyerInfo.get("group") )  and isValidInput( buyerInfo.get("is_married") )  and isValidInput( buyerInfo.get("no_child") )  and isValidInput(houseInfo.get("no_bedroom") )
Break rule code:  A
	Code of group B  | Group condition: buyerInfo.get("avai_amt") is not None and buyerInfo.get("monthly_amt") is not None
{'rule_group': 'B', 'rule_code': 'B1', 'score': 1.0, 'rule_expression': 'houseInfo.get("price") >= 0.9 * buyerInfo.get("avai_amt") and houseInfo.get("price") <= buyerInfo.get("avai_amt")', 'rule_desc': 'Nếu giá của ngôi nhà bằng khoảng 0.9-1 số tiền người mua hiện có', 'rule_expression_backup': 'houseInfo.get("price") >= 0.8 * buyerInfo.get("avai_amt")'}
0
{'rule_group': 'B', 'rule_

Hàm executeCase: duyệt từng BĐS và xác định mực độ phù hợp của các BĐS

In [7]:
def executeCase(custNo=0, outputFileLocation=curDir + '\\testOutput\\cust_6'):
    csv_file = outputFileLocation + '.csv'
    xlsx_file = outputFileLocation + '.xlsx'
    
    if os.path.exists(csv_file):
        os.remove(csv_file)
    if os.path.exists(xlsx_file):
        os.remove(xlsx_file)
    
    _housePerformanceList=[] 
    for i in houses:
        _houseEvaluation= calculateScoreOfHouse(buyers[custNo], i, rulesDf, ruleGroups, isDebugRule= 0)
        _housePerformanceList.append(_houseEvaluation)
    _housePrfmDf= pd.DataFrame.from_dict(_housePerformanceList)
    _housePrfmDf.to_csv(outputFileLocation +'.csv', index=False, quotechar= '"' )
    _housePrfmDf.to_excel(outputFileLocation+'.xlsx')
    return _housePerformanceList


## Execute Test case

## Hướng dẫn
1. Các function đã được định nghĩa  
1. Tất cả dữ liệu, bộ rule, và dữ liệu về KH được lưu trong file --> update dựa trên file excel này  
1. Chạy các test case KH bằng hàm executeCase với tham số là số thứ tự KH tương ứng trong excel và vị trí file output

### Test case for Complex rules

In [8]:
# loadData()
# # Done case X1
# print(buyers[15])
# case_a1= executeCase(custNo= 15, outputFileLocation=curDir + '\\testOutput\\cust_x1')

### Test case for No_Bed_Room rules

In [9]:
# loadData()
# print(buyers[4])
# case_a1= executeCase(custNo= 4, outputFileLocation='testOutput\\cust_a1')
# print(buyers[5])
# case_a2= executeCase(custNo= 5, outputFileLocation='testOutput\\cust_a2')
# print(buyers[6])
# case_a3= executeCase(custNo= 6, outputFileLocation='testOutput\\cust_a3')


### Test case for LOCATION rules

In [10]:
# loadData()
# print(buyers[11])
# case_c1= executeCase(custNo= 11, outputFileLocation='testOutput\\cust_c1')
# print(buyers[12])
# case_c2= executeCase(custNo= 12, outputFileLocation='testOutput\\cust_c2')
# print(buyers[13])
# case_c3= executeCase(custNo= 13, outputFileLocation='testOutput\\cust_c3')


### Test case for PAPER rules

In [11]:
# loadData()
# print(buyers[14])
# case_d1= executeCase(custNo= 14, outputFileLocation='testOutput\\cust_d1')

### Test case for FIN rules

In [12]:
# loadData()
# print(buyers[7])
# case_b1= executeCase(custNo= 7, outputFileLocation='testOutput\\cust_b1')
# print(buyers[8])
# case_b2= executeCase(custNo= 8, outputFileLocation='testOutput\\cust_b2')
# print(buyers[9])
# case_b3= executeCase(custNo= 9, outputFileLocation='testOutput\\cust_b3')
# print(buyers[10])
# case_b4= executeCase(custNo= 10, outputFileLocation='testOutput\\cust_b4')


In [13]:
# loadData()
# print(buyers[15])
# case_x1 = executeCase(custNo=15, outputFileLocation='testOutput\\cust_x1')
# print(buyers[16])
# case_x2 = executeCase(custNo=16, outputFileLocation='testOutput\\cust_x2')

Create csv and xlsx for x1

In [14]:
# loadData()
# print(buyers[17])
# case_x1 = executeCase(custNo=17, outputFileLocation=curDir + '\\testOutput\\cust_x1')

Create csv and xlsx for x2

In [15]:
# loadData()
# print(buyers[18])
# case_x2 = executeCase(custNo=18, outputFileLocation=curDir + '\\testOutput\\cust_x2')

Create csv and xlsx for x2

In [16]:
loadData()
print(buyers[18])
case_x2 = executeCase(custNo=18, outputFileLocation=curDir + '\\testOutput\\cust_x2')

{'id': 'x2', 'group': 1.0, 'age': 30.0, 'is_married': 0.0, 'no_child': 0.0, 'month_income': 30000000.0, 'monthly_amt': 40000000.0, 'avai_amt': 3000000000.0, 'desired_location': 'Quận 10', 'desired_interiorStatus': nan}
